<a href="https://colab.research.google.com/github/Deepanshu41008/AVSE/blob/main/squeezeformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install squeezeformer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [1]:
#https://github.com/upskyy/Squeezeformer
#og code upar hai, edited part neeche, according to the part ki apan daalenge aidio+video mix tokens in the squeezeformer kyuki squeezeformers works better on audio tokens

In [3]:
import torch
import torch.nn as nn
from squeezeformer.model import Squeezeformer

In [4]:

BATCH_SIZE = 4
SEQ_LENGTH = 500
INPUT_SIZE = 80
NUM_CLASSES = 10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


criterion = nn.CTCLoss().to(device)


model = Squeezeformer(num_classes=NUM_CLASSES).to(device)


input_lengths = torch.IntTensor([500, 450, 400, 350]).to(device)

targets = torch.LongTensor([
    [1, 3, 3, 3, 3, 3, 4, 5, 6, 2],
    [1, 3, 3, 3, 3, 3, 4, 5, 2, 0],
    [1, 3, 3, 3, 3, 3, 4, 2, 0, 0],
    [1, 3, 3, 3, 3, 3, 6, 2, 0, 0]
]).to(device)
target_lengths = torch.LongTensor([9, 8, 7, 7]).to(device)

audio_tokens = torch.randn(BATCH_SIZE, SEQ_LENGTH, INPUT_SIZE).to(device)
video_tokens = torch.randn(BATCH_SIZE, SEQ_LENGTH, INPUT_SIZE).to(device)


fused_tokens = torch.cat([audio_tokens, video_tokens], dim=1)
fused_lengths = input_lengths * 2


for _ in range(3):
    outputs, output_lengths = model(fused_tokens, fused_lengths)

    loss = criterion(outputs.transpose(0, 1), targets[:, 1:], output_lengths, target_lengths)
    loss.backward()
    print(f"Loss: {loss.item():.4f}")

Loss: 48.5595
Loss: 48.9348
Loss: 48.1841
